## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [1]:
NAME = "Dimitris Ntiniaropoulos"
AEM = "----"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [2]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [3]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x2001baeb5e0>)

In [4]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

In all cells below, the best option out of all tested is the one that is uncommented.

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models  

In [5]:
# BEGIN CODE HERE
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold , cross_validate,  GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, StackingClassifier, AdaBoostClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier

train_set = pd.read_csv("train_set.csv").sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

# Set one
cls1 = SVC(probability = False ,random_state=RANDOM_STATE)
cls2 = SVC(gamma='auto', probability = False ,random_state=RANDOM_STATE) 
cls3 = SVC(kernel= 'linear', probability = False ,random_state=RANDOM_STATE)
classifiers = [('svm1', cls1),('svm2', cls2),('svm3', cls3)]

# Set two
# cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE)
# cls2 = DecisionTreeClassifier(max_depth=16, random_state=RANDOM_STATE)
# cls3 = DecisionTreeClassifier(max_depth=8, random_state=RANDOM_STATE)
# classifiers = [('dt1', cls1),('dt2', cls2),('dt3', cls3)]

# Set three
# cls1 = KNeighborsClassifier(n_neighbors=5)
# cls2 = KNeighborsClassifier(n_neighbors=7)
# cls3 = KNeighborsClassifier(n_neighbors=9)
# classifiers = [('knn1', cls1),('knn2', cls2),('knn3', cls3)]

# Set four
# cls1 = SVC(probability=False, random_state=RANDOM_STATE)
# cls2 = DecisionTreeClassifier(random_state=RANDOM_STATE)
# cls3 = KNeighborsClassifier(n_neighbors=5)
# classifiers = [('svm', cls1),('dt', cls2),('knn', cls3)]

soft_vcls = VotingClassifier(estimators=classifiers, voting='soft')
hard_vcls = VotingClassifier(estimators=classifiers, voting='hard')

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

svlcs_scores = cross_validate(soft_vcls, X, y, scoring=('accuracy', 'f1'), cv=cv, n_jobs=-1)
s_avg_fmeasure = np.mean(svlcs_scores['test_f1']) # The average f-measure
s_avg_accuracy = np.mean(svlcs_scores['test_accuracy']) # The average accuracy

hvlcs_scores = cross_validate(hard_vcls, X, y, scoring=('accuracy', 'f1'), cv=cv, n_jobs=-1)
h_avg_fmeasure = np.mean(hvlcs_scores['test_f1']) # The average f-measure
h_avg_accuracy = np.mean(hvlcs_scores['test_accuracy']) # The average accuracy
#END CODE HERE

In [6]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('svm1', SVC(random_state=42)),
                             ('svm2', SVC(gamma='auto', random_state=42)),
                             ('svm3', SVC(kernel='linear', random_state=42))],
                 voting='soft')
F1 Weighted-Score: nan & Balanced Accuracy: nan


You should achive above 82% (Soft Voting Classifier)

In [7]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('svm1', SVC(random_state=42)),
                             ('svm2', SVC(gamma='auto', random_state=42)),
                             ('svm3', SVC(kernel='linear', random_state=42))])
F1 Weighted-Score: 0.8807 & Balanced Accuracy: 0.8572


You should achieve above 80% in both! (Hard Voting Classifier)

Random combinations of estimators were tested, wit set one being the best option.

| Set | Soft Voting F1 | Soft Voting Accuracy | Hard Voting F1 | Hard Voting Accuracy |
|-----|----------------|----------------------|----------------|----------------------|
| 1   | -              | -                    | 0.8807         | 0.8572               |
| 2   | 0.7619         | 0.7152               | 0.7620         | 0.7152               |
| 3   | 0.8384         | 0.8091               | 0.8384         | 0.8091               |
| 4   | -              | -                    | 0.8602         | 0.8327               |

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




In [8]:
# BEGIN CODE HERE

# Set one
# cls1 = SVC(probability = False ,random_state=RANDOM_STATE)
# cls2 = SVC(gamma='auto', probability = False ,random_state=RANDOM_STATE)
# cls3 = "" # Classifier #3 (Optional)

# classifiers = [('svm1',cls1),('svm2',cls2)]

# Set two
# cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE)
# cls2 = DecisionTreeClassifier(max_depth=16, random_state=RANDOM_STATE)
# cls3 = "" # Classifier #3 (Optional)

# classifiers = [('dt1',cls1),('dt2',cls2)]

# Set three
# cls1 = KNeighborsClassifier(n_neighbors=5)
# cls2 = KNeighborsClassifier(n_neighbors=7)
# cls3 = "" # Classifier #3 (Optional)

# classifiers = [('knn1',cls1),('knn2',cls2)]

# Set four
cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE)
cls2 = SVC(probability=False, random_state=RANDOM_STATE)
cls3 = "" # Classifier #3 (Optional)

classifiers = [('dt',cls1),('svm',cls2)]

# Set five
# cls1 = KNeighborsClassifier(n_neighbors=5)
# cls2 = SVC(probability=False, random_state=RANDOM_STATE)
# cls3 = "" # Classifier #3 (Optional)

# classifiers = [('knn',cls1),('svm',cls2)]

scls = StackingClassifier(classifiers, cv=10, passthrough=False) # Stacking Classifier

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
scores = cross_validate(scls, X, y, scoring=('accuracy', 'f1'), cv=cv, verbose=2, n_jobs=-1)

avg_fmeasure = np.mean(scores['test_f1']) # The average f-measure
avg_accuracy = np.mean(scores['test_accuracy']) # The average accuracy
#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 29.3min remaining: 29.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 29.9min finished


In [9]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(cv=10,
                   estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                               ('svm', SVC(random_state=42))])
F1 Weighted Score: 0.8778 & Balanced Accuracy: 0.8556


You should achieve above 85% in both

In this case, set four gave us the best results.

| Set | F1 Score | Accuracy |
|-----|----------|----------|
| 1   | 0.8768   | 0.8544   |
| 2   | 0.7696   | 0.7159   |
| 3   | 0.8405   | 0.8072   |
| 4   | 0.8778   | 0.8556   |
| 5   | 0.8767   | 0.8541   |

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [10]:
# BEGIN CODE HERE
ens1 = BaggingClassifier(DecisionTreeClassifier(max_depth=12), max_features=0.8, n_estimators=100, random_state=RANDOM_STATE)
ens2 = RandomForestClassifier(criterion='gini', max_depth=10, max_features='auto', n_estimators=500, random_state=RANDOM_STATE)
ens3 = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=100, random_state=RANDOM_STATE)
tree = DecisionTreeClassifier(criterion='gini',max_depth=7, min_samples_leaf=2,splitter='best', random_state=RANDOM_STATE)

f_measures = dict()
accuracies = dict()
# Example f_measures = {'Simple Decision': 0.8551, 'Ensemble with random ...': 0.92, ...}

# A couple of functions to help us
def evaluation(model, X, y):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_validate(model, X, y, scoring=('accuracy', 'f1'), cv=cv, verbose=2, n_jobs=-1)
    return scores

def scoring(model, X, y):
    scores = evaluation(model, X, y)
    f1_score = np.mean(scores['test_f1'])  
    acc_score = np.mean(scores['test_accuracy'])
    return f1_score, acc_score

f1_ens1, acc_ens1 = scoring(ens1, X, y)
f1_ens2, acc_ens2 = scoring(ens2, X, y)
f1_ens3, acc_ens3 = scoring(ens3, X, y)
f1_tree, acc_tree = scoring(tree, X, y)

f_measures['Bagging'] = f1_ens1
f_measures['RandomForest'] = f1_ens2
f_measures['GradientBoost'] = f1_ens3
f_measures['DecisionTree'] = f1_tree

accuracies['Bagging'] = acc_ens1
accuracies['RandomForest'] = acc_ens2
accuracies['GradientBoost'] = acc_ens3
accuracies['DecisionTree'] = acc_tree

#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  5.3min remaining:  5.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  5.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   27.5s remaining:   27.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  3.3min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    3.9s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.0s finished


In [11]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,round(score,4)))

BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=12),
                  max_features=0.8, n_estimators=100, random_state=42)
RandomForestClassifier(max_depth=10, n_estimators=500, random_state=42)
GradientBoostingClassifier(max_depth=5, random_state=42)
DecisionTreeClassifier(max_depth=7, min_samples_leaf=2, random_state=42)
Classifier:Bagging -  F1 Weighted:0.8463
Classifier:RandomForest -  F1 Weighted:0.8362
Classifier:GradientBoost -  F1 Weighted:0.86
Classifier:DecisionTree -  F1 Weighted:0.77
Classifier:Bagging -  BalancedAccuracy:0.8086
Classifier:RandomForest -  BalancedAccuracy:0.7886
Classifier:GradientBoost -  BalancedAccuracy:0.8296
Classifier:DecisionTree -  BalancedAccuracy:0.7204


**2.2** Describe your classifiers and your results.

Gradient Boost gave the best results.

| Classifier     | F1 Score | Accuracy |
|----------------|----------|----------|
| Bagging        | 0.8463   | 0.8086   |
| Random Forest  | 0.8362   | 0.7886   |
| Gradient Boost | 0.8600   | 0.8296   |
| Decision Tree  | 0.7700   | 0.7204   |

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

In a bagging classifier, we can have the estimators run in parallel so that's how the training time can be decreased. Unfortunately, in a boosting classifier the estimators can't be ran in parallel, because each has to take into account the previous one's mistakes.

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [12]:
# BEGIN CODE HERE

# Set one
# ens1 = SVC(C=10, gamma=0.0001, probability = False ,random_state=RANDOM_STATE)
# ens2 = SVC(C=1, gamma=0.01, probability = False ,random_state=RANDOM_STATE)
# ens3 = KNeighborsClassifier(n_neighbors=8,weights='distance' )
# ens4 = KNeighborsClassifier(n_neighbors=15,weights='distance' )
# ens5 = DecisionTreeClassifier(criterion='entropy',max_depth=15,min_samples_leaf=50)
# ens6 = DecisionTreeClassifier(criterion='gini',max_depth=8,min_samples_leaf=5)

# classifiers = [('svm1',ens1),('svm2',ens2),('knn1',ens3),('knn2',ens4),('dt1',ens5),('dt2',ens6)]

# Set two
ens1 = RandomForestClassifier(criterion='gini', max_depth=8, max_features='auto', n_estimators=500, random_state=RANDOM_STATE)
ens2 = KNeighborsClassifier(n_neighbors=8, weights='distance')
ens3 = KNeighborsClassifier(n_neighbors=15)
ens4 = SVC(C= 0.1, gamma=1, kernel='poly', probability = False ,random_state=RANDOM_STATE)
ens5 = SVC(C=10, gamma=0.0001, probability = False ,random_state=RANDOM_STATE)

classifiers = [('rf',ens1),('knn1',ens2),('knn2',ens3),('svm1',ens4),('svm2',ens5)]

# Set three
# ens1 = KNeighborsClassifier(n_neighbors=8, weights='distance')
# ens2 = KNeighborsClassifier(n_neighbors=15)
# ens3 = SVC(C= 0.1, gamma=1, kernel='poly', probability = False ,random_state=RANDOM_STATE)
# ens4 = SVC(C=10, gamma=0.0001, probability = False ,random_state=RANDOM_STATE)
# ens5 = RandomForestClassifier(criterion='gini', max_depth=8, max_features='auto', n_estimators=500, random_state=RANDOM_STATE)
# ens6 = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=100, random_state=RANDOM_STATE)

# classifiers = [('knn1',ens1),('knn2',ens2),('svm1',ens3),('svm2',ens4),('rf',ens5),('gb',ens6)]

best_cls = StackingClassifier(classifiers, cv=10, n_jobs=1)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
scores_stacking = cross_validate(best_cls, X, y, scoring=('accuracy', 'f1'), cv=cv, verbose=2, n_jobs=-1)

best_fmeasure = np.mean(scores_stacking['test_f1'])     
best_accuracy = np.mean(scores_stacking['test_accuracy'])
#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 59.8min remaining: 59.8min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 59.9min finished


In [13]:
print("Classifier:")
print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
StackingClassifier(cv=10,
                   estimators=[('rf',
                                RandomForestClassifier(max_depth=8,
                                                       n_estimators=500,
                                                       random_state=42)),
                               ('knn1',
                                KNeighborsClassifier(n_neighbors=8,
                                                     weights='distance')),
                               ('knn2', KNeighborsClassifier(n_neighbors=15)),
                               ('svm1',
                                SVC(C=0.1, gamma=1, kernel='poly',
                                    random_state=42)),
                               ('svm2',
                                SVC(C=10, gamma=0.0001, random_state=42))],
                   n_jobs=1)
F1 Weighted-Score:0.885870956173741 & Balanced Accuracy:0.8648698715763155


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

As in the first sextion of the assignment, various random combinations were tested.

| Set | F1 Score | Accuracy |
|-----|----------|----------|
| 1   | 0.88534  | 0.86415  |
| 2   | 0.88587  | 0.86486  |
| 3   | 0.88567  | 0.86463  |

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [14]:
# BEGIN CODE HERE
cls = best_cls
#END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
cls.fit(X, y)
predictions = cls.predict(test_set)
predictions_list = list(predictions)
print(cls)
print(predictions)

StackingClassifier(cv=10,
                   estimators=[('rf',
                                RandomForestClassifier(max_depth=8,
                                                       n_estimators=500,
                                                       random_state=42)),
                               ('knn1',
                                KNeighborsClassifier(n_neighbors=8,
                                                     weights='distance')),
                               ('knn2', KNeighborsClassifier(n_neighbors=15)),
                               ('svm1',
                                SVC(C=0.1, gamma=1, kernel='poly',
                                    random_state=42)),
                               ('svm2',
                                SVC(C=10, gamma=0.0001, random_state=42))],
                   n_jobs=1)
[1 0 1 ... 1 1 1]


LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [15]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!